In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import nltk #Natural language tool kit
import ast
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
Movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [4]:
Movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
Movies.shape

(4803, 20)

In [6]:
credits.shape

(4803, 4)

In [7]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
# now we are combing the data of movies and credits for are ease
Movies=Movies.merge(credits,on='title')

In [9]:
Movies.head(1)
# the attributes are neccessary for recommendation system we only use that

# genres
# id->movie_id
# keyword
# original language
# title
# overview
# cast
# crew->as it has the director name so it is important.

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


# PRE-PROCESSING

In [10]:
# new attributes ->table is
Movies=Movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [11]:
Movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [12]:
import ast

In [13]:
def covert(text):
    L=[]
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [14]:
Movies.dropna(inplace=True)

In [15]:
#till we convert genres attribute from dict of list -> only List
Movies['genres']=Movies['genres'].apply(covert)
Movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [16]:
#cast attribute fillteration
def convert3(text):
    L=[]
    counter=0
    for i in ast.literal_eval(text):
        if counter>3:
            L.append(i['name'])
        counter+=1
    return L

In [17]:
Movies['cast']=Movies['cast'].apply(convert3)

In [18]:
Movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[Michelle Rodriguez, Giovanni Ribisi, Joel Dav...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[Chow Yun-fat, Bill Nighy, Geoffrey Rush, Jack...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[Monica Bellucci, Ben Whishaw, Naomie Harris, ...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[Tom Hardy, Marion Cotillard, Joseph Gordon-Le...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[Thomas Haden Church, Mark Strong, Ciarán Hind...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [19]:
#cast attribute fillteration
Movies['cast']=Movies['cast'].apply(lambda x:x[0:3])

In [20]:
#function to fetch director name from cast
def fetch_director(text):
    L=[]
    for i in ast.literal_eval(text):
        if i['job']=='Director':
            L.append(i['name'])
    return L

In [21]:
Movies['crew']=Movies['crew'].apply(fetch_director)

In [22]:
Movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[Michelle Rodriguez, Giovanni Ribisi, Joel Dav...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[Chow Yun-fat, Bill Nighy, Geoffrey Rush]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[Monica Bellucci, Ben Whishaw, Naomie Harris]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[Tom Hardy, Marion Cotillard, Joseph Gordon-Le...",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[Thomas Haden Church, Mark Strong, Ciarán Hinds]",[Andrew Stanton]


In [23]:
#overview attributes data is in list to we are converting into 
#list so it will ease while merging the attributes
Movies['overview']=Movies['overview'].apply(lambda x:x.split())

In [24]:
Movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[Michelle Rodriguez, Giovanni Ribisi, Joel Dav...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[Chow Yun-fat, Bill Nighy, Geoffrey Rush]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[Monica Bellucci, Ben Whishaw, Naomie Harris]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[Tom Hardy, Marion Cotillard, Joseph Gordon-Le...",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[Thomas Haden Church, Mark Strong, Ciarán Hinds]",[Andrew Stanton]


In [25]:
# now remove the WHITE spaces b/w the character so that are model 
# did not confuse with similar names   eg: Sam Mendes->SamMendes
Movies['genres']=Movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
Movies['keywords']=Movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
Movies['cast']=Movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
Movies['crew']=Movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [26]:
Movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[[, {, "", i, d, "", :, , 1, 4, 6, 3, ,, , "", n,...","[MichelleRodriguez, GiovanniRibisi, JoelDavidM...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[[, {, "", i, d, "", :, , 2, 7, 0, ,, , "", n, a,...","[ChowYun-fat, BillNighy, GeoffreyRush]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[[, {, "", i, d, "", :, , 4, 7, 0, ,, , "", n, a,...","[MonicaBellucci, BenWhishaw, NaomieHarris]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[[, {, "", i, d, "", :, , 8, 4, 9, ,, , "", n, a,...","[TomHardy, MarionCotillard, JosephGordon-Levitt]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[[, {, "", i, d, "", :, , 8, 1, 8, ,, , "", n, a,...","[ThomasHadenChurch, MarkStrong, CiaránHinds]",[AndrewStanton]


In [27]:
#concatination into tags->overview,cast,crew,keyword
Movies['tags']=Movies['overview']+Movies['genres']+Movies['keywords']+Movies['cast']+Movies['crew']

In [28]:
#new data frame so that we ease in use
new_df=Movies[['movie_id','title','tags']]

In [29]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_8268\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [30]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [31]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction [ { " i d " :  1 4 6 3 ,  " n a m e " :  " c u l t u r e  c l a s h " } ,  { " i d " :  2 9 6 4 ,  " n a m e " :  " f u t u r e " } ,  { " i d " :  3 3 8 6 ,  " n a m e " :  " s p a c e  w a r " } ,  { " i d " :  3 3 8 8 ,  " n a m e " :  " s p a c e  c o l o n y " } ,  { " i d " :  3 6 7 9 ,  " n a m e " :  " s o c i e t y " } ,  { " i d " :  3 8 0 1 ,  " n a m e " :  " s p a c e  t r a v e l " } ,  { " i d " :  9 6 8 5 ,  " n a m e " :  " f u t u r i s t i c " } ,  { " i d " :  9 8 4 0 ,  " n a m e " :  " r o m a n c e " } ,  { " i d " :  9 8 8 2 ,  " n a m e " :  " s p a c e " } ,  { " i d " :  9 9 5 1 ,  " n a m e " :  " a l i e n " } ,  { " i d " :  1 0 1 4 8 ,  " n a m e " :  " t r i b e " } ,  { " i d " :  1 0 1 5 8 ,  " n a m e " :  " a l i e n  p l a n e t " } ,  

In [32]:
#convert data into lower alphabet
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_8268\1303414833.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [33]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


# VECTORIZATION

In [34]:
import nltk

#importing lib. named as PORTERSTEMMER->which help us to convert repating data into single one

In [35]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [36]:
#from sklearn library we are using sklearn.feature_extraction.text->importing COUNT VECTORIZATION
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [37]:
#function
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [38]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_8268\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [39]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [40]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [41]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [42]:
cv.get_feature_names_out()
#len(cv.get_feature_names_out())#->it consist of 5000 words that came most frequently 

array(['000', '007', '10', ..., 'zombies', 'zone', 'zoo'], dtype=object)

->so the issue/problem is duplicant of words like 
action
actions , this are similar words that create an issue so we have to merge these words into ONE. 
all ACTIONS -> transfromed into ACTION.

['Love','Loved','Loving']
->['Love','Love','Love']

CALCULATING THE SIMILARITES

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
similarity=cosine_similarity(vectors)

In [45]:
similarity[1]

array([0.14638501, 1.        , 0.08728716, ..., 0.03217447, 0.        ,
       0.03407991])

# MODEL

In [46]:
enumerate(similarity[0])

In [47]:
list(enumerate(similarity[0]))  #here each and every index gets the distance with all movies.

[(0, 1.0000000000000002),
 (1, 0.14638501094227996),
 (2, 0.08944271909999159),
 (3, 0.11359236684941296),
 (4, 0.14509525002200233),
 (5, 0.15118578920369088),
 (6, 0.061429511683395124),
 (7, 0.1917412472118426),
 (8, 0.08770580193070293),
 (9, 0.14907119849998596),
 (10, 0.14704292441876154),
 (11, 0.13333333333333333),
 (12, 0.1538967528127731),
 (13, 0.07071067811865475),
 (14, 0.20080483222562473),
 (15, 0.10476454436543672),
 (16, 0.1267731382092775),
 (17, 0.13046561461068845),
 (18, 0.053838190205816545),
 (19, 0.10882143751650175),
 (20, 0.08199200616907877),
 (21, 0.08032193289024989),
 (22, 0.10846522890932808),
 (23, 0.14301938838683884),
 (24, 0.08606629658238704),
 (25, 0.026352313834736497),
 (26, 0.1690308509457033),
 (27, 0.17616606585441102),
 (28, 0.158113883008419),
 (29, 0.09682458365518543),
 (30, 0.10741723110591492),
 (31, 0.23094010767585027),
 (32, 0.06523280730534423),
 (33, 0.16269784336399212),
 (34, 0.0),
 (35, 0.16770509831248423),
 (36, 0.19764235376052

In [48]:
#sorting
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]  

[(2999, 0.2795084971874737),
 (4405, 0.25819888974716115),
 (61, 0.24999999999999997),
 (1444, 0.23904572186687872),
 (529, 0.2333333333333333)]

In [49]:
#main function
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    #printing the list
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        #print(i[0])

In [50]:
recommend('Gandhi')#here the index of five movies get printed

Gandhi, My Father
Ramanujan
Mr. Turner
The Wind That Shakes the Barley
Guiana 1838


In [51]:
import pickle


In [53]:
#we are converting our data frame into dictonary
new_df.to_dict()

{'movie_id': {0: 19995,
  1: 285,
  2: 206647,
  3: 49026,
  4: 49529,
  5: 559,
  6: 38757,
  7: 99861,
  8: 767,
  9: 209112,
  10: 1452,
  11: 10764,
  12: 58,
  13: 57201,
  14: 49521,
  15: 2454,
  16: 24428,
  17: 1865,
  18: 41154,
  19: 122917,
  20: 1930,
  21: 20662,
  22: 57158,
  23: 2268,
  24: 254,
  25: 597,
  26: 271110,
  27: 44833,
  28: 135397,
  29: 37724,
  30: 558,
  31: 68721,
  32: 12155,
  33: 36668,
  34: 62211,
  35: 8373,
  36: 91314,
  37: 68728,
  38: 102382,
  39: 20526,
  40: 49013,
  41: 44912,
  42: 10193,
  43: 534,
  44: 168259,
  45: 72190,
  46: 127585,
  47: 54138,
  48: 81005,
  49: 64682,
  50: 9543,
  51: 68726,
  52: 38356,
  53: 217,
  54: 105864,
  55: 62177,
  56: 188927,
  57: 10681,
  58: 5174,
  59: 14161,
  60: 17979,
  61: 76757,
  62: 258489,
  63: 411,
  64: 246655,
  65: 155,
  66: 14160,
  67: 15512,
  68: 1726,
  69: 44826,
  70: 8487,
  71: 1735,
  72: 297761,
  73: 2698,
  74: 137113,
  75: 9804,
  76: 14869,
  77: 150540,
  78:

In [54]:
#convert dict. data into pickle formate
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [55]:
#dumping the simliarity file
pickle.dump(similarity,open('similarity.pkl','wb'))